# Building a House (Problem in the Harvard Business Review, 1963)

Several tasks must be completed in order to build our house. Each task has a duration, and some tasks may be worked on simultaneously, but there are also task dependencies that induce precedence constraints. Some tasks can only be started once other tasks have been completed. The following table shows each task, its duration, and the tasks that must be completed before it starts. How fast can the house be built?

![alt text](https://hbr.org/resources/images/article_assets/hbr/6309/63508_B.gif)

Let's model finding the critical (longest) path in this graph this as a network flow problem.

In [ ]:
#Reuse the same data as last time

#Array for storing task labels (:a, :b, ..., :x)
tasks = []
for i = 'a':'x'
    #Can convert back to string using string(sym), e.g. string(:hello) returns "hello"
    push!(tasks, Symbol(i))
end

#Dictionary to store the task durations
dur = [0, 4, 2, 4, 6, 1, 2, 3, 2, 4, 10, 3, 1, 2, 3, 2, 1, 1, 2, 3, 1, 2, 5, 0]
duration = Dict(zip(tasks,dur))

#Dictionary to store immediate predecessors for each project
predecessors = ( [], [:a], [:b], [:c], [:d], [:c], [:f], [:f], [:d], [:d,:g], [:i,:j,:h], [:k],
    [:l], [:l], [:l], [:e], [:p], [:c], [:o,:t], [:m,:n], [:t], [:q,:r], [:v], [:s,:u,:w])
pred = Dict(zip(tasks,predecessors));

#Initialize and build edge set, store edge costs
edges = []
costs = []

#Create an edge for each task predecessor, starting from predecessor and ending at task, with cost equal to task's duration
for i in tasks
    for j in pred[i]
        push!(edges, (j,i))
        push!(costs, duration[i])
    end
end

#Store edge costs in dictionary with edges as keys
edge_costs = Dict(zip(edges,costs));

#Initialize incidence matrix
using NamedArrays

zero_matrix = zeros( size(tasks,1), size(edges,1) )
A = NamedArray( zero_matrix, (tasks,edges), ("task","edge") )

#Build incidence matrix column-by-column
for e in edges
    A[e[1],e] = 1  #in column for edge e, put a 1 in the row corresponding to the task that the edge starts from
    A[e[2],e] = -1  #in column for edge e, put a -1 in the row corresponding to the task that the edge ends at
end

#Node supplies: 1 for task :a, -1 for task :x, 0 for all others
b = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1];

In [ ]:
using JuMP
using HiGHS

m = Model(HiGHS.Optimizer)

#Edges from node j to node i
@variable(m, 0 <= x[edges] <= 1)

@constraint(m, flow_conservation, A*x .== b)

@objective(m, Max, sum(edge_costs[e]*x[e] for e in edges))

print(m)

In [ ]:
optimize!(m)

In [ ]:
@show objective_value(m)

for e in edges
    if(value(x[e]) > 10^-9)
        println("The edge from ", e[1], " to ", e[2], " is part of the longest path.")
    end
end